In [50]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [51]:
df1 = pd.read_csv('data1.csv')
df2 = pd.read_csv('data2.csv')
df3 = pd.read_csv('data3.csv')
data = pd.concat([df1, df2, df3], ignore_index=True)

In [52]:
data = data.drop_duplicates()

In [53]:
for i in data[['price', 'lowest_price', 'highest_price']]:
    data[i] = data[i].str.replace('руб', '').str.replace('\xa0', '').str.replace(' ', '')
    

In [54]:
for i in data[['price', 'lowest_price', 'highest_price']]:
    data[i] = np.where(data[i].str.isnumeric(), data[i], np.nan)
    data[i] = data[i].astype(float).astype('Int64')

for i in data[['rating', 'stars']]:
    data[i] = data[i].str.replace(',', '.').astype(float)

In [55]:
data.to_csv('data.csv', index=False)

In [56]:
top_50 = data.groupby('city').apply(lambda x: x.nlargest(50, 'rating')).reset_index(drop=True)
top_50.to_csv('top_50.csv', index=False)

In [57]:
average_df = data.groupby('city').agg({'price': 'mean', 'rating': 'mean', 'stars': 'mean', 'lowest_price': 'mean', 'highest_price': 'mean'}).reset_index().round(2)
average_df.to_csv('average.csv', index=False)

In [58]:
average_df

,city,price,rating,stars,lowest_price,highest_price
0,Абу-Даби,18303.42,4.31,4.25,12737.01,22920.66
1,Аланья,6184.71,4.09,4.01,6203.38,22943.75
2,Алма-Ата,8978.38,3.87,3.75,6824.16,8686.99
3,Астана,12700.33,4.02,3.65,4857.21,6522.76
4,Афины,13724.45,4.20,3.37,8370.7,15814.96
5,Бангкок,12904.2,4.19,3.27,4312.44,6000.85
6,Барселона,21655.0,4.13,3.08,14238.31,24980.84
7,Батуми,7077.57,4.26,NaN,3819.64,5469.89
8,Берлин,11799.6,3.97,3.38,8858.0,14292.74
9,Бирмингем,8122.56,3.85,3.13,7497.76,12476.31


In [59]:
url = 'https://time-in.ru/coordinates?ysclid=ltsiewolv228461763'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html')
names = soup.find_all('ul', class_ = 'coordinates-items')

In [60]:
coordinates = pd.DataFrame()

for i in names[1]:      
   
    match = re.search(r'([А-Яа-яЁё\-]+)(-?\d+\.\d+),\s*(-?\d+\.\d+)', i.text)
    if match != None:
        name = match.group(1) 
        if name in average_df['city'].unique():
            latitude = match.group(2)
            longitude = match.group(3)
            df = pd.DataFrame({
            'city': [name],
            'lat': [latitude],
            'lon': [longitude]
            })

            coordinates = pd.concat([coordinates, df], ignore_index=True)

In [61]:
more_c = []
for i in average_df['city']:
    if i not in coordinates['city'].unique():
        more_c.append(i)


In [62]:
more = ['36.5438, 31.9998', '43.2567, 76.9286', '41.3888, 2.15899', '41.6423, 41.6339',
        '52.4814, -1.89983', '42.29115, 18.8403', '-34.6132, -58.3772', '49.2497, -123.119',
        '5.9667, 80.4167', '45.4371, 12.3327', '62.094444, 7.080556', '40.7942, 43.8453',
        '25.0657, 55.1713', '-35.2835, 149.128', '7.2955, 80.6356', '50.2327, 12.8712', 
        '35.3417, 33.3167', '67.85, 20.217', '34.0522, -118.244', '43.297, 5.38107',
        '-32.8908, -68.8272', '48.1374, 11.5755', '34.6851, 135.805', '40.7143, -74.006',
        '35.1028, 129.04', '7.89059, 98.3981', '36.4356, 28.222', '36.7658, 31.3842',
        '-33.8679, 151.207', '41.9283, 19.2064', '27.2574, 33.8129']

In [63]:
lat_more = []
lon_more = []
for i in more:
    lat_more.append(i.split(',')[0])
    lon_more.append(i.split(',')[1])

In [64]:
more_coordinates = pd.DataFrame({
    'city': more_c,
    'lat': lat_more,
    'lon': lon_more
})

In [65]:
coordinates = pd.concat([coordinates, more_coordinates], ignore_index=True)

In [66]:
coordinates

,city,lat,lon
0,Абу-Даби,24.47,54.37
1,Астана,51.18,71.45
2,Афины,37.98,23.73
3,Бангкок,13.75,100.5
4,Берлин,52.52,13.41
5,Ереван,40.18,44.51
6,Каир,30.06,31.25
7,Лондон,51.51,-0.13
8,Мадрид,40.42,-3.7
9,Никосия,35.18,33.36


In [67]:
gen_all = pd.read_csv('gen_all.csv')

In [68]:
gen_all.head()

,Unnamed: 0,country,city,"insurance, руб.","ticket_price, руб.",luggage_cost,flight_time_minutes,rest_rating,rest_price,hotel_rating,hotel_price,stars_grade,transport_grade,cards_grade,lang_grade,rest_rating_grade,rest_price_grade,hotel_rating_grade,hotel_price_grade
0,0,Армения,Ереван,1801,14803.8,5827.888889,1183.0,4.345882,5434.777228,4.25,6767.05,3.52,5,5,3,3,2,4,1
1,1,Австралия,Сидней,9171,69848.0,7370.900000,3170.0,4.274790,2373.470790,3.90,17745.44,3.65,4,1,2,2,4,1,1
2,2,Австралия,Канберра,7884,106518.1,5300.800000,3451.0,3.910204,1683.957237,3.91,10477.33,3.97,4,1,2,1,5,1,1
3,3,Англия,Бирмингем,45170,32343.6,9588.100000,2170.0,4.315657,2051.748387,3.85,8122.56,3.13,3,3,2,3,4,1,1
4,4,Англия,Лондон,45170,27551.6,13269.800000,1774.5,4.582215,3004.180723,3.98,19789.40,3.49,5,3,2,4,3,1,1


In [69]:
gen_all = gen_all[['country', 'city', 'insurance, руб.', 'ticket_price, руб.', 'luggage_cost', 'flight_time_minutes', 'rest_rating', 'rest_price', 'hotel_rating']]

In [70]:
gen_all['lowest_hotel_price'] = average_df['lowest_price']
gen_all['highest_hotel_price'] = average_df['highest_price']

In [71]:
df_coordinates = pd.merge(gen_all, coordinates, on='city')

In [72]:
df_coordinates.to_csv('new_coordinates.csv', index=False)

In [73]:
gen_all.head()

,country,city,"insurance, руб.","ticket_price, руб.",luggage_cost,flight_time_minutes,rest_rating,rest_price,hotel_rating,lowest_hotel_price,highest_hotel_price
0,Армения,Ереван,1801,14803.8,5827.888889,1183.0,4.345882,5434.777228,4.25,12737.01,22920.66
1,Австралия,Сидней,9171,69848.0,7370.900000,3170.0,4.274790,2373.470790,3.90,6203.38,22943.75
2,Австралия,Канберра,7884,106518.1,5300.800000,3451.0,3.910204,1683.957237,3.91,6824.16,8686.99
3,Англия,Бирмингем,45170,32343.6,9588.100000,2170.0,4.315657,2051.748387,3.85,4857.21,6522.76
4,Англия,Лондон,45170,27551.6,13269.800000,1774.5,4.582215,3004.180723,3.98,8370.7,15814.96
